# yo 

One of the approach for solving the problem statements which were:
1. Problem Transformation:
These approaches involve transforming a multi-label classification problem into a
variant or series of single-label problems.
Eg- Binary Relevance, Label Powersets, Classification Chains to

Let us import all the libraries which are required 

In [75]:
import pandas as pd
import numpy as np
import ast
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.multiclass import OneVsRestClassifier
from sklearn.pipeline import Pipeline

from sklearn.multioutput import ClassifierChain
from sklearn.metrics import accuracy_score, hamming_loss, f1_score

# importing
the dataset , which was merged into one , to make my work easier

In [76]:
df=pd.read_csv(r"C:\RKT\CODE\python\projects\assignment\Data\final_movie_genre_v2.csv")
# Display first few rows
print("Original Data:")
display(df.head())

Original Data:


,title,overview,genre_ids,genres
0,The Shawshank Redemption,Imprisoned in the 1940s for the double murder ...,"[18, 80]","['Drama', 'Crime']"
1,The Godfather,"Spanning the years 1945 to 1955, a chronicle o...","[18, 80]","['Drama', 'Crime']"
2,The Godfather Part II,In the continuing saga of the Corleone crime f...,"[18, 80]","['Drama', 'Crime']"
3,Schindler's List,The true story of how businessman Oskar Schind...,"[18, 36, 10752]","['Drama', 'History', 'War']"
4,12 Angry Men,The defense and the prosecution have rested an...,[18],['Drama']


Well weLL we dont need overview as well as genre ids , we are gonna drop em 

In [ ]:
df = df.dropna(subset=['overview'])
#string to list
df['genres_list'] = df['genres'].apply(ast.literal_eval)


we are using multilabelbinarizer , well in smple words , think of it as a one hot encoding but with multiple winners of a independent variable 


In [78]:
mlb = MultiLabelBinarizer()
y = mlb.fit_transform(df['genres_list'])

the exportion of this MultiLabelBinarizer was done later (below is the code)

In [106]:
import joblib
joblib.dump(mlb, 'mlb_classes.pkl')

['mlb_classes.pkl']

lets get the unique number of genres we have on our hand 

In [79]:
print(f"\nUnique Classes ({len(mlb.classes_)}): {mlb.classes_}")
print(f"Shape of Y: {y.shape}")


Unique Classes (18): ['Action' 'Adventure' 'Animation' 'Comedy' 'Crime' 'Drama' 'Family'
 'Fantasy' 'History' 'Horror' 'Music' 'Mystery' 'Romance'
 'Science Fiction' 'TV Movie' 'Thriller' 'War' 'Western']
Shape of Y: (9999, 18)


Vectorizer , cuz of course the machine cant understand words and we also need to give importance score to words ,rather than
converts a collection of raw documents into a numerical matrix based on word importance.

In [80]:
tfidf = TfidfVectorizer(max_features=10000, stop_words='english')
X = tfidf.fit_transform(df['title'])

similarly 
the exportion of this vectorizer  was done later (below is the code)

In [105]:
import joblib

joblib.dump(tfidf, 'tfidf_vectorizer.pkl')

['tfidf_vectorizer.pkl']

In [81]:
print(f"Shape of X: {X.shape}")

Shape of X: (9999, 7126)


our traintest slpit 

In [82]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

well we are gonna go for binary relevance and logistics regression 
binary relevance  in simple one , will say like , "Hey are you action" yes or no .........hey are you drama yes or no ...
goin to each genre one by one 
 

 for logistics regresison it give me a prbabilities for the classes and well it is fast as compared training a NN 
 and loooking the dataset , this was the first algo came to my mind

In [83]:
br_lr = OneVsRestClassifier(
    LogisticRegression(
        solver="liblinear",max_iter=100
    )
)

br_lr.fit(X_train, y_train)
y_pred_br_lr = br_lr.predict(X_test)


The evaluation ,as usual , but heres the catch , weneed hamming loss to be lowered and sub accuracy to be higee, based ont this ,we weill mostly desgin 

In [84]:
print("Binary Relevance + Logistic Regression")
print("Hamming Loss:", hamming_loss(y_test, y_pred_br_lr))
print("Micro F1:", f1_score(y_test, y_pred_br_lr, average="micro"))
print("Macro F1:", f1_score(y_test, y_pred_br_lr, average="macro"))
print("Exact Match Accuracy:", accuracy_score(y_test, y_pred_br_lr))


Binary Relevance + Logistic Regression
Hamming Loss: 0.1366388888888889
Micro F1: 0.22180034804619522
Macro F1: 0.10346711997922431
Exact Match Accuracy: 0.0455


# ML FLow

this was kinda new to me , it took me a time to learn , thank you for this , i got to learn to implement it 

we are logging the model, its parameters and its metrics 

In [107]:
import mlflow
import mlflow.sklearn
from sklearn.metrics import accuracy_score, classification_report

mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("movie_predictor_V2")

with mlflow.start_run(run_name="Logistic_Regression_and binary_relevance"):

    acc = accuracy_score(y_test, y_pred_br_lr)
    report = classification_report(y_test, y_pred_br_lr, output_dict=True)

    mlflow.log_params({"solver": "liblinear", "max_iter": 100})


    #   metrics
    mlflow.log_metrics({
        "accuracy": acc,
        "recall_class_0": report["0"]["recall"],
        "recall_class_1": report["1"]["recall"],
        "f1_macro": report["macro avg"]["f1-score"]
    })
  
    mlflow.sklearn.log_model(
        sk_model=br_lr,
        artifact_path="model",
        registered_model_name="LogisticRegressionModelBinaryReL"
    )


c:\Users\Boondi ka Ladoo\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\Boondi ka Ladoo\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
2026/02/10 21:03:46 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
c:\Users\Boondi ka Ladoo\AppData\Local\Programs\Python\Python311\Lib\site-packages\mlflow\models\model.py:1209: FutureWarning: Saving scikit-learn models in the pick

🏃 View run Logistic_Regression_and binary_relevance at: http://127.0.0.1:5000/#/experiments/809687645743597975/runs/c7933f8ccbf84c8e837f5f951e8ab506
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/809687645743597975


Created version '1' of model 'LogisticRegressionModelBinaryReL'.


# lets get to another method 

classification chain and linear svm

Classification chain , lets call it CC , it removes the bad thing that was about binary rrelevance , which was ignoring the relationship betweent the classes , for example a movie can be action and adventourus , 

SVMs are famous for handling a huge number of features without getting "confused" or slowing down as much as other models.

it is better for multilabel text 

In [86]:
from sklearn.svm import LinearSVC

base_svm = LinearSVC()

chain_svm = ClassifierChain(base_svm)
chain_svm.fit(X_train, y_train)

y_pred_chain_svm = chain_svm.predict(X_test)


In [87]:
print("\nClassifier Chain + Linear SVM")
print("Hamming Loss:", hamming_loss(y_test, y_pred_chain_svm))
print("Micro F1:", f1_score(y_test, y_pred_chain_svm, average="micro"))
print("Macro F1:", f1_score(y_test, y_pred_chain_svm, average="macro"))
print("Exact Match Accuracy:", accuracy_score(y_test, y_pred_chain_svm))



Classifier Chain + Linear SVM
Hamming Loss: 0.13747222222222222
Micro F1: 0.4419889502762431
Macro F1: 0.31386890632982456
Exact Match Accuracy: 0.1055


this was kinda new to me , it took me a time to learn , thank you for this , i got to learn to implement it 

we are logging the model, its parameters and its metrics 

In [ ]:
mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("movie_predictor_V2")

with mlflow.start_run(run_name="Classification chain and linearSVM"):

    acc = accuracy_score(y_test, y_pred_chain_svm)
    report = classification_report(y_test, y_pred_chain_svm, output_dict=True)




    #   metrics
    mlflow.log_metrics({
        "accuracy": acc,
        "recall_class_0": report["0"]["recall"],
        "recall_class_1": report["1"]["recall"],
        "f1_macro": report["macro avg"]["f1-score"]
    })
  
    mlflow.sklearn.log_model(
        sk_model=chain_svm,
        artifact_path="model_chain_linear_svm",
        registered_model_name="ClassificationchainandlinearSVM"
    )

c:\Users\Boondi ka Ladoo\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
2026/02/10 21:04:37 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
c:\Users\Boondi ka Ladoo\AppData\Local\Programs\Python\Python311\Lib\site-packages\mlflow\models\model.py:1209: FutureWarning: Saving scikit-learn models in the pickle or cloudpickle format requires exercising caution because these formats rely on Python's object serialization mechanism, which can execute arbitrary code during deserialization.The recommended safe alternative is the 'skops' format.
  flavor.save_model(path=local_path, mlflow_model=mlflow_model, **kwargs)
Successfully registered model 'Classificationchainandl

🏃 View run Classification chain and linearSVM at: http://127.0.0.1:5000/#/experiments/809687645743597975/runs/fb2d214478cc4241b519736cf87418a8
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/809687645743597975


Created version '1' of model 'ClassificationchainandlinearSVM'.


In [92]:
print("TF-IDF features:", tfidf.transform(sample_title).shape)


TF-IDF features: (1, 7126)


this was me , testing one of the models , which quite worked 

i had to lower the thresshold , because we didnt took overview as a column , so the model didint hadnt much to learn 


In [102]:
# BR and Lr

import numpy as np

sample_title = ["Rush Hour"]

sample_X = tfidf.transform(sample_title)

probs = br_lr.predict_proba(sample_X)[0]

threshold = 0.3

predicted_genres = [
    mlb.classes_[i]
    for i, p in enumerate(probs)
    if p >= threshold
]

print("Predicted Genres:", predicted_genres)


Predicted Genres: ['Action', 'Comedy', 'Crime', 'Drama']


as u can see the model is performing good 

In [99]:
sample_title = ["The Dark Knight"]

sample_X = tfidf.transform(sample_title)

# Predict
sample_pred = chain_svm.predict(sample_X)

predicted_genres = mlb.inverse_transform(sample_pred)

print("Predicted Genres:", predicted_genres)


Predicted Genres: [('Action', 'Adventure', 'Science Fiction')]


i had to do the same for CC+ SVM

In [ ]:
import numpy as np

sample_title = ["Interstellar"]

# Transform input  TF-IDF
sample_X = tfidf.transform(sample_title)
# SVM outputs decision scores, which are mapped to genres using a custom threshold:
scores = chain_svm.decision_function(sample_X)[0]

threshold = -0.3   # try -0.3, -0.5, -1.0 if needed

# Convert scores to genre labels
predicted_genres = [
    mlb.classes_[i]
    for i, score in enumerate(scores)
    if score >= threshold
]

print("Predicted Genres:", predicted_genres)


Predicted Genres: ['Adventure', 'Drama', 'Science Fiction']


In [109]:
import joblib

joblib.dump(chain_svm, 'chain_svm.pkl')

['chain_svm.pkl']

as u can see classification chain is perfomring better than binary relevance

we will be loading that model 